In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
from tqdm import trange
from operator import itemgetter
import numpy as np
from pprint import pprint

import matplotlib.pyplot as plt
from matplotlib import rc
gridc = (1., 1., 1)
plt.rcParams['grid.color'] = gridc
plt.rcParams["axes.edgecolor"] = (0.898, 0.925, 0.965, 1)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

from funcs import DataBase

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>div#site { height: 100% !important; }</style>"))

In [ ]:
db = DataBase('../assets/iclr2023.db')
db.initialize(create=False)

# Rating distribution bar

In [ ]:
# all decisions w/o withdrawn
_cmd = "SELECT s_4_avg FROM submissions WHERE withdraw == 0;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
act_avgs = np.array(data)
print("> Total submissions: {}".format(len(data)))
print(f"    Average ratings: {act_avgs.mean():.2f}")
print(f"    Max ratings: {act_avgs.max():.2f}")
print(f"    Min ratings: {act_avgs.min():.2f}")

# withdraw
_cmd = "SELECT s_4_avg FROM submissions WHERE withdraw == 1;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
wd_avgs = np.array(data)
print("> Reject submissions: {}".format(wd_avgs.shape[0]))
print(f"    Average ratings: {wd_avgs.mean():.2f}")
print(f"    Max ratings: {wd_avgs.max():.2f}")
print(f"    Min ratings: {wd_avgs.min():.2f}")


In [ ]:
width = 0.3
fig = plt.figure(figsize=[16, 6])

ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor((0.898, 0.925, 0.965, 0.5))
ax.spines['left'].set_color('w')
ax.spines['bottom'].set_color('w')
ax.spines['right'].set_color('w')
ax.spines['top'].set_color('w')

_min, _max = act_avgs.min(), act_avgs.max()
hist, bin_edges = np.histogram(act_avgs, bins=20, range=(_min, _max))

# all
ax.bar(np.linspace(_min, _max, len(hist)), hist, width=width, alpha=0.95,
       color='#789BFF', capsize=4, label='Active')

# reject
hist_reject, _ = np.histogram(wd_avgs, bins=20, range=(_min, _max))
ax.bar(np.linspace(_min, _max, len(hist)), hist_reject, width=width, alpha=0.95,
       color='#ffccbd', capsize=4, label='Withdraw')

for i, v in zip(np.linspace(_min, _max, len(hist)), hist):
    ax.text(i - 0.15 if v >= 100 else i - 0.1, v + 10, str(v), color='#1f59ff', fontsize=16)

ax.text(1, 500, f"Average rating: {act_avgs.mean():.2f}", color='#1f59ff', fontsize=20)
plt.ylim(0, 800)
plt.xticks(ticks=np.linspace(_min, _max, len(hist)), rotation=40, labels=[f"{d:.1f}" for d in np.linspace(_min, _max, len(hist))])
ax.set_ylabel(r"# submissions", fontsize=20)
ax.set_xlabel("Rating", fontsize=14)
ax.set_axisbelow(True)
ax.grid()
ax.legend(loc=1, fontsize=18)
plt.savefig('../images/stats_bar_1204.png', dpi=150)

# Writing html

In [ ]:
_cmd = "SELECT url_id, title, s_0_avg, s_0_std, s_0_cnt, s_0_list, s_1_avg, s_1_std, s_1_cnt, s_1_list FROM submissions WHERE withdraw == 0 ORDER BY s_0_avg DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

for i, d in enumerate(data[1]):
    print(f"{i}--------{d}")

In [ ]:
# _cmd = "SELECT url_id, title, s_0_avg, s_0_std, s_0_cnt, s_0_list FROM submissions WHERE withdraw == 0 ORDER BY s_0_avg DESC;"
_cmd = "SELECT url_id, title, s_0_avg, s_0_std, s_0_cnt, s_0_list, s_4_avg, s_4_std, s_4_cnt, s_4_list FROM submissions WHERE withdraw == 0 ORDER BY s_0_avg DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

# read template 
with open('../iclr2023_submissions.template.html', 'r') as f:
    html_temp = f.readlines()
# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1

# decision map
decision_map = {
    'oral': 2,
    'spotlight': 1,
    'poster': 0,
}

# write data
for i in trange(len(data)):
    _data = data[i]
#     decision = _data[8].split("(")[-1].split(')')[0]
    diff = _data[6] - _data[2]
    diff_cls = 'none'
    if diff > 0:
        diff_cls = 'positive'
    if diff < 0:
        diff_cls = 'negative'
    _str = f"<tr><td>{i + 1}</td><td class='td-left'><a href='https://openreview.net/forum?id={_data[0]}'> {_data[1]}</a></td>" \
           f"<td>{_data[2]:.2f}</td>" \
           f"<td>{_data[6]:.2f}</td>" \
           f"<td>{_data[7]:.2f}</td>" \
           f"<td><span class='{diff_cls}'>{diff:.2f}</td>" \
           f"<td>" \
           f"<table class='sub-table'>" \
           f"<tr><td class='r1'>{', '.join([str(int(float(_d))) for _d in _data[5].split(', ')][::-1])}</td></tr>" \
           f"<tr><td class='r2'>{', '.join([str(int(float(_d))) for _d in _data[9].split(', ')][::-1])}</td></tr>" \
           f"</table></td>" \
           f"</tr>\n"
#            "<td data-sort='{decision_map[decision.lower()]}' class='{decision.lower()}'>{decision}</td>" \
    html_temp.insert(idx + i, _str)

with open('../iclr2023_submissions.max.html' ,'w') as f:
    f.write("".join(html_temp))

### key words

In [ ]:
# all submissions 
_cmd = "SELECT keywords FROM submissions;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
keywords = {}
for i in trange(len(data)):
    _kw = data[i][0].split(', ')
    _kw = [_k.lower().strip() for _k in _kw]
    for _k in _kw:
        if _k in keywords.keys():
            keywords[_k] += 1
        else:
            keywords[_k] = 1
# sort values
keywords = {k: v for k, v in sorted(keywords.items(), key=lambda item: item[1])[::-1]}

### statistics bar

In [ ]:
# all decisions
_cmd = "SELECT rating_0_avg, rating_3_avg FROM submissions;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
rating_avgs = np.array(data).transpose()
print("\n> Total submissions (including the withdraw): {}".format(len(data)))
print(f"  Avg: {rating_avgs[1].mean():.2f} | Max: {rating_avgs[1].max():.2f} | Min: {rating_avgs[1].min():.2f} | ΔR: {(rating_avgs[1] - rating_avgs[0]).mean():.2f}")

# oral
_cmd = "SELECT rating_0_avg, rating_3_avg FROM submissions WHERE decision LIKE '%Oral%';"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
oral_avgs = np.array(data).transpose()
print("\n> Oral submissions: {}".format(oral_avgs[0].shape))
print(f"  Avg: {oral_avgs[1].mean():.2f} | Max: {oral_avgs[1].max():.2f} | Min: {oral_avgs[1].min():.2f} | ΔR: {(oral_avgs[1] - oral_avgs[0]).mean():.2f}")

# spotlight
_cmd = "SELECT rating_0_avg, rating_3_avg FROM submissions WHERE decision LIKE '%Spotlight%';"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
spotlight_avgs = np.array(data).transpose()
print("\n> Soitlight submissions: {}".format(spotlight_avgs[0].shape))
print(f"  Avg: {spotlight_avgs[1].mean():.2f} | Max: {spotlight_avgs[1].max():.2f} | Min: {spotlight_avgs[1].min():.2f} | ΔR: {(spotlight_avgs[1] - spotlight_avgs[0]).mean():.2f}")

# poster
_cmd = "SELECT rating_0_avg, rating_3_avg FROM submissions WHERE decision LIKE '%Poster%';"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
poster_avgs = np.array(data).transpose()
print("\n> Poster submissions: {}".format(poster_avgs[0].shape))
print(f"  Avg: {poster_avgs[1].mean():.2f} | Max: {poster_avgs[1].max():.2f} | Min: {poster_avgs[1].min():.2f} | ΔR: {(poster_avgs[1] - poster_avgs[0]).mean():.2f}")

# all accepted
_cmd = "SELECT rating_0_avg, rating_3_avg FROM submissions WHERE withdraw == 0 AND decision != 'Reject';"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
acpt_avgs = np.array(data).transpose()
print("\n> Accepted submissions: {}".format(acpt_avgs[0].shape))
print(f"  Avg: {acpt_avgs[1].mean():.2f} | Max: {acpt_avgs[1].max():.2f} | Min: {acpt_avgs[1].min():.2f} | ΔR: {(acpt_avgs[1] - acpt_avgs[0]).mean():.2f}")

# reject
_cmd = "SELECT rating_0_avg, rating_3_avg FROM submissions WHERE decision LIKE '%Reject%';"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
reject_avgs = np.array(data).transpose()
print("\n> Reject submissions: {}".format(reject_avgs[0].shape))
print(f"  Avg: {reject_avgs[1].mean():.2f} | Max: {reject_avgs[1].max():.2f} | Min: {reject_avgs[1].min():.2f} | ΔR: {(reject_avgs[1] - reject_avgs[0]).mean():.2f}")

# all
_cmd = "SELECT rating_0_avg, rating_3_avg FROM submissions WHERE withdraw == 0;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
all_avgs = np.array(data).transpose()
print("\n> All submissions: {}".format(all_avgs[0].shape))
print(f"  Avg: {all_avgs[1].mean():.2f} | Max: {all_avgs[1].max():.2f} | Min: {all_avgs[1].min():.2f} | ΔR: {(all_avgs[1] - all_avgs[0]).mean():.2f}")

In [ ]:
np.where(all_avgs[1] - all_avgs[0] < 0)[0].shape

In [ ]:
_cmd = "SELECT rating_3_avg FROM submissions WHERE withdraw == 0;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()
rating_avgs = np.array(data)
_min, _max = rating_avgs.min(), rating_avgs.max()

width = 0.24
fig = plt.figure(figsize=[16, 6], frameon=False)

ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor((0.898, 0.925, 0.965, 0.5))
ax.spines['left'].set_color('w')
ax.spines['bottom'].set_color('w')
ax.spines['right'].set_color('w')
ax.spines['top'].set_color('w')

# all submissions
hist, bin_edges = np.histogram(rating_avgs, bins=20, range=(_min, _max))
# ax.bar(np.linspace(_min, _max, len(hist)), hist, width=width, alpha=0.95, 
#        color='#789BFF', capsize=4)
print(sum(hist))
for i, v in zip(np.linspace(_min, _max, len(hist)), hist):
    ax.text(i - 0.05 * len(str(v)), v + 6.0, str(v), color='#2f3a49', fontsize=16)

# reject
hist_reject, _ = np.histogram(reject_avgs, bins=20, range=(_min, _max))
print(f"Rejct: {sum(hist_reject)}")
ax.bar(np.linspace(_min, _max, len(hist)), hist_reject, width=width, alpha=0.95, 
       color='#B6C3FF', capsize=4, label=f'{"Reject":-<14}avg: {reject_avgs.mean():.2f}')

# poster
hist_poster, _ = np.histogram(poster_avgs, bins=20, range=(_min, _max))
print(f"Poster: {sum(hist_poster)}")
ax.bar(np.linspace(_min, _max, len(hist)), hist_poster, bottom=hist_reject, width=width, alpha=0.95, 
       color='#ffced6', capsize=4, label=f'{"Poster":-<14}avg: {poster_avgs.mean():.2f}')

# spotlight
hist_spotlight, _ = np.histogram(spotlight_avgs, bins=20, range=(_min, _max))
print(f"Spotlight: {sum(hist_spotlight)}")
ax.bar(np.linspace(_min, _max, len(hist)), hist_spotlight, bottom=hist_poster + hist_reject, 
       width=width, alpha=0.95, 
       color='#FF95A4', capsize=4, label=f'{"Spotlight":-<14}avg: {spotlight_avgs.mean():.2f}')

# oral
hist_oral, _ = np.histogram(oral_avgs, bins=20, range=(_min, _max))
print(f"Oral: {sum(hist_oral)}")
ax.bar(np.linspace(_min, _max, len(hist)), hist_oral, bottom=hist_poster + hist_reject + hist_spotlight, 
       width=width, alpha=0.95, 
       color='#FF2542', capsize=4, label=f'{"Oral":-<14}avg: {oral_avgs.mean():.2f}')

plt.ylim(0, 450)
plt.xticks(ticks=np.linspace(_min, _max, len(hist)), 
           rotation=40, 
           labels=[f"{d:.2f}" for d in np.linspace(_min, _max, len(hist))])
ax.set_ylabel(r"# submissions", fontsize=14)
ax.set_xlabel("Rating", fontsize=14)
ax.set_axisbelow(True)
ax.grid()
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc=2, fontsize=14)
plt.title('ICLR 2022 Rating Distribution')
plt.savefig('../images/stats_bar_0130.png')

### write all submissions

In [ ]:
_cmd = "SELECT id, url, title, rating_0_avg, rating_3_avg, ratings_0, ratings_3, rating_3_cnt, decision FROM submissions WHERE withdraw == 0 ORDER BY rating_3_avg DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

for i, d in enumerate(data[1]):
    print(f"{i}--------{d}")

In [ ]:
_cmd = "SELECT id, url, title, rating_0_avg, rating_3_avg, ratings_0, ratings_3, rating_3_cnt, decision FROM submissions WHERE withdraw == 0 AND decision!= 'Reject' ORDER BY rating_3_avg DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

# read template 
with open('../iclr2022_submissions_template.html', 'r') as f:
    html_temp = f.readlines()
# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1

# decision map
decision_map = {
    'oral': 2,
    'spotlight': 1,
    'poster': 0,
}

# write data
for i in trange(len(data)):
    _data = data[i]
    decision = _data[8].split("(")[-1].split(')')[0]
    
    _str = f"<tr><td>{i + 1}</td><td class='td-left'><a href='{_data[1]}'> {_data[2]}</a></td>" \
           f"<td class='r1'>{_data[3]:.2f}</td><td>{_data[4]:.2f}</td>" \
           f"<td data-sort='{(5.0 + _data[4] - _data[3]):.2f}'>{(_data[4]-_data[3]):.2f}</td>" \
           f"<td data-sort='{_data[7]}'>" \
           f"<table class='sub-table'>" \
           f"<tr><td class='r1'>{', '.join([str(int(float(_d))) for _d in _data[5].split(', ')][::-1])}</td>" \
           f"</tr><tr><td class='r2'>{', '.join([str(int(float(_d))) for _d in _data[6].split(', ')][::-1])}</td></tr>" \
           f"</table></td><td data-sort='{decision_map[decision.lower()]}' class='{decision.lower()}'>{decision}</td>" \
           f"</tr>\n"
    html_temp.insert(idx + i, _str)

with open('../iclr2022_submissions.html' ,'w') as f:
    f.write("".join(html_temp))

### write rejected html

In [ ]:
_cmd = "SELECT id, url, title, rating_0_avg, rating_3_avg, ratings_0, ratings_3, rating_3_cnt, decision FROM submissions WHERE withdraw == 0 AND decision == 'Reject' ORDER BY rating_3_avg DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

# read template 
with open('../iclr2022_submissions_template.html', 'r', encoding='utf-8') as f:
    html_temp = f.readlines()
# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1

# write data
for i in trange(len(data)):
    _data = data[i]
    decision = _data[8].split("(")[-1].split(')')[0]
    
    _str = f"<tr><td>{i + 1}</td><td class='td-left'><a href='{_data[1]}'> {_data[2]}</a></td>" \
           f"<td class='r1'>{_data[3]:.2f}</td><td>{_data[4]:.2f}</td>" \
           f"<td data-sort='{(5.0 + _data[4] - _data[3]):.2f}'>{(_data[4]-_data[3]):.2f}</td>" \
           f"<td data-sort='{_data[7]}'>" \
           f"<table class='sub-table'>" \
           f"<tr><td class='r1'>{', '.join([str(int(float(_d))) for _d in _data[5].split(', ')][::-1])}</td>" \
           f"</tr><tr><td class='r2'>{', '.join([str(int(float(_d))) for _d in _data[6].split(', ')][::-1])}</td></tr>" \
           f"</table></td><td class='{decision.lower()}'>{decision}</td>" \
           f"</tr>\n"
    html_temp.insert(idx + i, _str)

with open('../iclr2022_submissions_reject.html' ,'w', encoding='utf-8') as f:
    f.write("".join(html_temp))

### write oral

In [ ]:
_cmd = "SELECT id, url, title, rating_0_avg, rating_3_avg, ratings_0, ratings_3, rating_3_cnt, decision FROM submissions WHERE withdraw == 0 AND decision LIKE '%Oral%' ORDER BY rating_3_avg DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

# read template 
with open('../iclr2022_submissions_template.html', 'r', encoding='utf-8') as f:
    html_temp = f.readlines()
# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1

# write data
for i in trange(len(data)):
    _data = data[i]
    decision = _data[8].split("(")[-1].split(')')[0]
    
    _str = f"<tr><td>{i + 1}</td><td class='td-left'><a href='{_data[1]}'> {_data[2]}</a></td>" \
           f"<td class='r1'>{_data[3]:.2f}</td><td>{_data[4]:.2f}</td>" \
           f"<td data-sort='{(5.0 + _data[4] - _data[3]):.2f}'>{(_data[4]-_data[3]):.2f}</td>" \
           f"<td data-sort='{_data[7]}'>" \
           f"<table class='sub-table'>" \
           f"<tr><td class='r1'>{', '.join([str(int(float(_d))) for _d in _data[5].split(', ')][::-1])}</td>" \
           f"</tr><tr><td class='r2'>{', '.join([str(int(float(_d))) for _d in _data[6].split(', ')][::-1])}</td></tr>" \
           f"</table></td><td class='{decision.lower()}'>{decision}</td>" \
           f"</tr>\n"
    html_temp.insert(idx + i, _str)

with open('../iclr2022_submissions_oral.html' ,'w', encoding='utf-8') as f:
    f.write("".join(html_temp))

### write spotlight

In [ ]:
_cmd = "SELECT id, url, title, rating_0_avg, rating_3_avg, ratings_0, ratings_3, rating_3_cnt, decision FROM submissions WHERE withdraw == 0 AND decision LIKE '%Spotlight%' ORDER BY rating_3_avg DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

# read template 
with open('../iclr2022_submissions_template.html', 'r', encoding='utf-8') as f:
    html_temp = f.readlines()
# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1

# write data
for i in trange(len(data)):
    _data = data[i]
    decision = _data[8].split("(")[-1].split(')')[0]
    
    _str = f"<tr><td>{i + 1}</td><td class='td-left'><a href='{_data[1]}'> {_data[2]}</a></td>" \
           f"<td class='r1'>{_data[3]:.2f}</td><td>{_data[4]:.2f}</td>" \
           f"<td data-sort='{(5.0 + _data[4] - _data[3]):.2f}'>{(_data[4]-_data[3]):.2f}</td>" \
           f"<td data-sort='{_data[7]}'>" \
           f"<table class='sub-table'>" \
           f"<tr><td class='r1'>{', '.join([str(int(float(_d))) for _d in _data[5].split(', ')][::-1])}</td>" \
           f"</tr><tr><td class='r2'>{', '.join([str(int(float(_d))) for _d in _data[6].split(', ')][::-1])}</td></tr>" \
           f"</table></td><td class='{decision.lower()}'>{decision}</td>" \
           f"</tr>\n"
    html_temp.insert(idx + i, _str)

with open('../iclr2022_submissions_spotlight.html' ,'w', encoding='utf-8') as f:
    f.write("".join(html_temp))

### write poster

In [ ]:
_cmd = "SELECT id, url, title, rating_0_avg, rating_3_avg, ratings_0, ratings_3, rating_3_cnt, decision FROM submissions WHERE withdraw == 0 AND decision LIKE '%Poster%' ORDER BY rating_3_avg DESC;"
db.cursor.execute(_cmd)
data = db.cursor.fetchall()

# read template 
with open('../iclr2022_submissions_template.html', 'r', encoding='utf-8') as f:
    html_temp = f.readlines()
# find insert index
idx = html_temp.index('    <!-- start here -->\n') + 1

# write data
for i in trange(len(data)):
    _data = data[i]
    decision = _data[8].split("(")[-1].split(')')[0]
    
    _str = f"<tr><td>{i + 1}</td><td class='td-left'><a href='{_data[1]}'> {_data[2]}</a></td>" \
           f"<td class='r1'>{_data[3]:.2f}</td><td>{_data[4]:.2f}</td>" \
           f"<td data-sort='{(5.0 + _data[4] - _data[3]):.2f}'>{(_data[4]-_data[3]):.2f}</td>" \
           f"<td data-sort='{_data[7]}'>" \
           f"<table class='sub-table'>" \
           f"<tr><td class='r1'>{', '.join([str(int(float(_d))) for _d in _data[5].split(', ')][::-1])}</td>" \
           f"</tr><tr><td class='r2'>{', '.join([str(int(float(_d))) for _d in _data[6].split(', ')][::-1])}</td></tr>" \
           f"</table></td><td class='{decision.lower()}'>{decision}</td>" \
           f"</tr>\n"
    html_temp.insert(idx + i, _str)

with open('../iclr2022_submissions_poster.html' ,'w', encoding='utf-8') as f:
    f.write("".join(html_temp))

In [ ]:
db.close()